<a href="https://colab.research.google.com/github/pranao0609/DL_Projects/blob/main/11_Character_Level_Language.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

In [13]:
text = "hello world this is a simple character level language model"

In [14]:
chars = sorted(list(set(text)))
char2idx = {ch: i for i, ch in enumerate(chars)}
idx2char = {i: ch for ch, i in char2idx.items()}

In [15]:
sequence = [char2idx[c] for c in text]

In [16]:
seq_len = 10
batch_size = 4

In [17]:
class CharDataset(Dataset):
    def __init__(self, sequence, seq_len):
        self.data = []
        for i in range(len(sequence) - seq_len):
            self.data.append((sequence[i:i+seq_len], sequence[i+seq_len]))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        x, y = self.data[idx]
        return torch.tensor(x), torch.tensor(y)

dataset = CharDataset(sequence, seq_len)
loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [18]:
class CharRNN(nn.Module):
    def __init__(self, vocab_size, hidden_size=64):
        super().__init__()
        self.embed = nn.Embedding(vocab_size, hidden_size)
        self.rnn = nn.RNN(hidden_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, vocab_size)

    def forward(self, x, h=None):
        x = self.embed(x)
        out, h = self.rnn(x, h)
        out = self.fc(out[:, -1, :])
        return out, h

model = CharRNN(len(chars))
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()

In [19]:
for epoch in range(9):
    for x, y in loader:
        out, _ = model(x)
        loss = criterion(out, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}")

Epoch 1, Loss: 2.8122
Epoch 2, Loss: 1.5425
Epoch 3, Loss: 0.7096
Epoch 4, Loss: 0.1300
Epoch 5, Loss: 0.1671
Epoch 6, Loss: 0.0877
Epoch 7, Loss: 0.0721
Epoch 8, Loss: 0.1015
Epoch 9, Loss: 0.0708


In [23]:
def generate_text(model, start_seq, length=50):
    model.eval()
    input_seq = torch.tensor([char2idx[c] for c in start_seq], dtype=torch.long).unsqueeze(0)
    h = None
    generated = start_seq
    for _ in range(length):
        out, h = model(input_seq, h)
        next_idx = torch.argmax(out, dim=1).item()
        next_char = idx2char[next_idx]
        generated += next_char
        input_seq = torch.cat([input_seq[:, 1:], torch.tensor([[next_idx]], dtype=torch.long)], dim=1)
    return generated


In [24]:
print("\nGenerated text:")
print(generate_text(model, start_seq="hello "))


Generated text:
hello this a simple character level language model langu
